In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/jovyan/__RHI/FINAL_PREPROCESSED_SKILLS_CLEANED.csv')

In [3]:
labeled_df = df.loc[df.hasSalary==True].reset_index(drop=True)

In [4]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
#from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

In [5]:
modelling_df = labeled_df.drop(columns=['url', 'title', 'location', 'salaryRange',
                                'maxSalary', 'currency', 'salaryPeriod', 'hasSalary',
                                          'company', 'position', 'datePosted', 'validThrough', 'jobDescription', 
                                           'skills', 'lang', 'titleLang', 'details'])

In [6]:
def filling_na(level):
    if pd.isna(level):
        return 'Unknown'
    else:
        return level

In [7]:
modelling_df['level'] = modelling_df['level'].apply(filling_na)

In [9]:
modelling_df.head()

,type,level,minSalary,.NET,3D Reconstruction,A/B testing,AGI,AI governance,AI strategy,AIStats,...,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy,finalJobDescription,scope
0,Internship,Entry,9000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,creator open source python library looking mul...,data science
1,Full Time,Senior,55000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,hfr wa founded specializes area hedge fundalte...,data science
2,Full Time,Senior,150000.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,usa remote position position summary data lead...,management
3,Full Time,Mid,84000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,position social science researcher senior mana...,management
4,Full Time,Mid,81600.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,building new group ambitious independentminded...,research science


In [10]:
X = modelling_df.drop('minSalary', axis = 1)
y = modelling_df.minSalary

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = ColumnTransformer([("poly", PolynomialFeatures(degree=2, interaction_only=True), slice(2, 298)),
                           ("tfidf", TfidfVectorizer(ngram_range=(1, 2)), 'finalJobDescription'),
                          ("cat_encoder", OneHotEncoder(sparse=False, drop='first'), ['level', 'type', 'scope'])
                          ], 
                          remainder='passthrough')


In [69]:
s = tfidf.fit_transform(X_train)

In [70]:
s.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [71]:
s_df = pd.DataFrame(s.toarray())

In [73]:
s_df[s_df[6]!=0]

,0,1,2,3,4,5,6,7,8,9,...,317041,317042,317043,317044,317045,317046,317047,317048,317049,317050


In [13]:
from tempfile import mkdtemp
from shutil import rmtree

In [14]:
from sklearn.neighbors import KNeighborsRegressor

In [15]:
from sklearn.model_selection import GridSearchCV, ParameterGrid

In [24]:
cachedir = mkdtemp() # creates temp directory

params = {'knn_reg__n_neighbors': [3, 5, 7], 
          'knn_reg__algorithm': ['brute'],
          'knn_reg__metric': ['cosine', 'euclidean', 'minkowski', 'canberra', 
                               'chebyshev', 'braycurtis', 'cityblock'],
          'preprocessor__tfidf__max_df': [0.95, 0.97, 1.0],
         #'tfidfv__min_df': [0.01, 0.02, 0.03]
         }

preprocessor = ColumnTransformer([("poly", PolynomialFeatures(degree=2, interaction_only=True), slice(2, 298)),
                           ("tfidf", TfidfVectorizer(ngram_range=(1, 2)), 'finalJobDescription'),
                          ("cat_encoder", OneHotEncoder(sparse=False, drop='first'), ['level', 'scope']),
                          ("type_encoder", OneHotEncoder(sparse=False, handle_unknown='ignore'), ['type']),        
                          ], 
                          remainder='passthrough')

knn_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('knn_reg', KNeighborsRegressor())], 
    memory=cachedir)

knn_cv = GridSearchCV(estimator=knn_pipe, param_grid=params, scoring=['neg_root_mean_squared_error', 
                                                                        'neg_mean_absolute_percentage_error'], 
                        cv=5, n_jobs=-1, verbose=1, refit='neg_root_mean_squared_error')

knn_cv.fit(X_train, y_train)

rmtree(cachedir) # removes temp directory

Fitting 5 folds for each of 63 candidates, totalling 315 fits


/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py:303: UserWarning: Persisting input arguments took 1.36s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py:303: UserWarning: Persisting input arguments took 1.43s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fi

In [25]:
pd.DataFrame(knn_cv.cv_results_)[['params', 'mean_test_neg_root_mean_squared_error', 
                                    'mean_test_neg_mean_absolute_percentage_error']].sort_values('mean_test_neg_mean_absolute_percentage_error', ascending=False).style.bar(vmin=0, vmax=1)

,params,mean_test_neg_root_mean_squared_error,mean_test_neg_mean_absolute_percentage_error
61,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 7, 'preprocessor__tfidf__max_df': 0.97}",-34358.534357,-0.299058
60,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 7, 'preprocessor__tfidf__max_df': 0.95}",-34382.111073,-0.299180
62,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 7, 'preprocessor__tfidf__max_df': 1.0}",-34354.622715,-0.299536
57,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 5, 'preprocessor__tfidf__max_df': 0.95}",-34213.589386,-0.301741
58,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 5, 'preprocessor__tfidf__max_df': 0.97}",-34198.401882,-0.301775
55,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 3, 'preprocessor__tfidf__max_df': 0.97}",-33793.735610,-0.301883
59,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 5, 'preprocessor__tfidf__max_df': 1.0}",-34185.808440,-0.302354
56,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 3, 'preprocessor__tfidf__max_df': 1.0}",-33806.763179,-0.303167
54,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'cityblock', 'knn_reg__n_neighbors': 3, 'preprocessor__tfidf__max_df': 0.95}",-33879.966264,-0.303179
15,"{'knn_reg__algorithm': 'brute', 'knn_reg__metric': 'euclidean', 'knn_reg__n_neighbors': 7, 'preprocessor__tfidf__max_df': 0.95}",-32564.824998,-0.339293


In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [26]:
rmse = mean_squared_error(y_test, knn_cv.predict(X_test), squared=False)
rmse

32268.21098612929

In [27]:
mape = mean_absolute_percentage_error(y_test, knn_cv.predict(X_test))
mape

0.3952825762606178

# Catboost

In [16]:
from catboost import CatBoostRegressor

cachedir = mkdtemp() # creates temp directory

params = {
          'cb_reg__iterations': [100, 150, 200],
          'cb_reg__learning_rate': [0.03, 0.05, 0.1],
          'cb_reg__depth': [2, 4, 6, 8],
          'cb_reg__l2_leaf_reg': [0.2, 0.5, 1, 3],
          'preprocessor__tfidf__max_df': [0.95, 0.97, 1.0],
         #'tfidfv__min_df': [0.01, 0.02, 0.03]
         }

preprocessor = ColumnTransformer([("poly", PolynomialFeatures(degree=2, interaction_only=True), slice(2, 298)),
                           ("tfidf", TfidfVectorizer(ngram_range=(1, 2)), 'finalJobDescription'),
                          ("cat_encoder", OneHotEncoder(sparse=False, drop='first'), ['level', 'scope']),
                          ("type_encoder", OneHotEncoder(sparse=False, handle_unknown='ignore'), ['type']),        
                          ], 
                          remainder='passthrough')

cb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('cb_reg', CatBoostRegressor(loss_function='RMSE'))], 
    memory=cachedir)

cb_cv = GridSearchCV(estimator=cb_pipe, param_grid=params, scoring=['neg_root_mean_squared_error', 
                                                                    'neg_mean_absolute_percentage_error'], 
                     cv=5, n_jobs=-1, verbose=1, refit='neg_root_mean_squared_error')

cb_cv.fit(X_train, y_train)

rmtree(cachedir) # removes temp directory

In [17]:
cachedir = mkdtemp() # creates temp directory

params = {
          'cb_reg__learning_rate': [0.03, 0.05, 0.1],
          'cb_reg__depth': [2, 4, 6, 8, 10],
          'cb_reg__l2_leaf_reg': [0.2, 0.5, 1, 3],
         }

preprocessor = ColumnTransformer([("poly", PolynomialFeatures(degree=2, interaction_only=True), slice(2, 298)),
                           ("tfidf", TfidfVectorizer(ngram_range=(1, 2), max_df=0.97), 'finalJobDescription'),
                          ("cat_encoder", OneHotEncoder(sparse=False, drop='first'), ['level', 'scope']),
                          ("type_encoder", OneHotEncoder(sparse=False, handle_unknown='ignore'), ['type']),        
                          ], 
                          remainder='passthrough')

cb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('cb_reg', CatBoostRegressor(iterations=100, loss_function='RMSE'))], 
    memory=cachedir)

cb_cv = GridSearchCV(estimator=cb_pipe, param_grid=params, scoring=['neg_root_mean_squared_error', 
                                                                    'neg_mean_absolute_percentage_error'], 
                     cv=5, n_jobs=-1, verbose=1, refit='neg_root_mean_squared_error')

cb_cv.fit(X_train, y_train)

rmtree(cachedir) # removes temp directory

Fitting 5 folds for each of 60 candidates, totalling 300 fits


/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py:303: UserWarning: Persisting input arguments took 1.36s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py:303: UserWarning: Persisting input arguments took 1.73s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fi

0:	learn: 34971.2264217	total: 291ms	remaining: 28.8s
1:	learn: 34632.0754922	total: 499ms	remaining: 24.5s
2:	learn: 34330.1138634	total: 744ms	remaining: 24.1s
3:	learn: 33979.0407208	total: 990ms	remaining: 23.8s
4:	learn: 33677.2748027	total: 1.22s	remaining: 23.1s
5:	learn: 33421.6337414	total: 1.49s	remaining: 23.3s
6:	learn: 33112.1490228	total: 1.71s	remaining: 22.7s
7:	learn: 32868.8162023	total: 1.92s	remaining: 22.1s
8:	learn: 32566.8580500	total: 2.14s	remaining: 21.6s
9:	learn: 32301.4238738	total: 2.4s	remaining: 21.6s
10:	learn: 32058.3186736	total: 2.63s	remaining: 21.3s
11:	learn: 31750.8028742	total: 2.84s	remaining: 20.8s
12:	learn: 31543.8362146	total: 3.1s	remaining: 20.8s
13:	learn: 31254.4862182	total: 3.34s	remaining: 20.5s
14:	learn: 30980.5041980	total: 3.6s	remaining: 20.4s
15:	learn: 30769.2155836	total: 3.82s	remaining: 20.1s
16:	learn: 30562.4246246	total: 4.11s	remaining: 20s
17:	learn: 30371.7222529	total: 4.33s	remaining: 19.7s
18:	learn: 30220.7434069	

exception calling callback for <Future at 0x7fe30d09ec10 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/opt/conda/lib/python3.9/site-packages/joblib/_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func))
  File "/opt/conda/lib/

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [25]:
params = dict(learning_rate=[.01, 0.05, .1, .15, .2, .25, .3, .5, 1],
              depth= [2, 4, 6, 8, 10],
              l2_leaf_reg= [0.2, 0.5, 1, 3])
                
grid = GridSearchCV(estimator=CatBoostRegressor(random_state=42, iterations=100, verbose=False),
                    param_grid=params,
                    scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                    cv=5,
                    refit='neg_root_mean_squared_error',
                    verbose=5
)

In [26]:
X_train_proc = preprocessor.fit_transform(X_train)

In [27]:
X_test_proc = preprocessor.transform(X_test)

In [ ]:
grid.fit(X_train_proc, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.353) neg_root_mean_squared_error: (test=-27054.929) total time=  31.5s
[CV 2/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.322) neg_root_mean_squared_error: (test=-24653.550) total time=  30.1s
[CV 3/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.329) neg_root_mean_squared_error: (test=-28168.259) total time=  30.4s
[CV 4/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.345) neg_root_mean_squared_error: (test=-28057.773) total time=  30.3s
[CV 5/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.360) neg_root_mean_squared_error: (test=-30638.301) total time=  30.6s
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.05; neg

In [ ]:
pd.DataFrame(grid.cv_results_)[['params', 'mean_test_neg_root_mean_squared_error', 
                                    'mean_test_neg_mean_absolute_percentage_error']].sort_values(
    'mean_test_neg_root_mean_squared_error', ascending=False).style.bar(vmin=0, vmax=1)

In [ ]:
rmse = mean_squared_error(y_test, grid.predict(X_test_proc), squared=False)
rmse

In [ ]:
mape = mean_absolute_percentage_error(y_test, grid.predict(X_test_proc))
mape

In [19]:
cb_cv.cv_results_

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
rmse = mean_squared_error(y_test, cb_cv.predict(X_test), squared=False)
rmse

In [ ]:
mape = mean_absolute_percentage_error(y_test, knn_cv.predict(X_test))
mape